In [ ]:
#!pip install transformers
#!pip install --upgrade unsloth unsloth_zoo

In [16]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import time
import os

#os.environ["PYTORCH_CUDA_ALLOC_CONF = expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
# --- 1. Configuration ---
# You can adjust these parameters
MODEL_NAME = "gpt2-xl"
MAX_LENGTH = 1024 # Maximum sequence length for GPT-2 Medium
BATCH_SIZE = 4
LEARNING_RATE = 5e-5
EPOCHS = 1
NUM_WORKERS = 4 # For faster data loading
PATH= 'MAAI_gpt2_extra_large_torch-50'

# --- 2. Device Setup (Essential for PyTorch) ---
# Automatically detects and uses GPU (CUDA/ROCm) or falls back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch using device: {device}")


PyTorch using device: cpu


In [17]:

# --- 5. Initialization ---

# Load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Set the padding token, which is necessary for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the Model and move it to the device (GPU or CPU)
#model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Use the lines below for loading stored weights and comment the line above
model = torch.load(PATH, weights_only=False,  map_location=torch.device('cpu'))



In [18]:
prompt = "तुमच्या"
max_new_tokens = 1024

# 1. Tokenize input and move to device
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)



In [15]:
# 2. Generate text
print(f"Generating continuation for prompt: '{prompt}'")
        # Use model.generate() for text generation
with torch.no_grad():
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=512, # Set max total length
    num_return_sequences=1,
    compile_config=None,
    do_sample=True, # Use sampling for more creative output
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id # Ensure generation stops correctly
    )

# 3. Decode and Print Output
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("-" * 20)
print(f"Generated Text:\n{generated_text}")
print("-" * 20)

Generating continuation for prompt: 'तुमच्या'


ValueError: You provided `compile_config` as an instance of <class 'NoneType'>, but it must be an instance of `CompileConfig`.

In [7]:
#PATH= 'gpt2_extra_large_50epoch_model'
#tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding= True)
#model = torch.load(PATH, weights_only=False)
prompt = "तुमच्या"
#prompt = "पंतप्रधान"
#prompt = "मी"
max_new_tokens = 1024

# 1. Tokenize input and move to device
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# 2. Generate text
print(f"Generating continuation for prompt: '{prompt}'")
        # Use model.generate() for text generation
with torch.no_grad():
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=1024, # Set max total length
    num_return_sequences=1,
    do_sample=True, # Use sampling for more creative output
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id # Ensure generation stops correctly
    )

# 3. Decode and Print Output
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("-" * 20)
print(f"Generated Text:\n{generated_text}")
print("-" * 20)

Generating continuation for prompt: 'तुमच्या'


ValueError: You provided `compile_config` as an instance of <class 'NoneType'>, but it must be an instance of `CompileConfig`.